I recently got familiar with the Apple ecosystem due to receiving a new iPhone from my mom. (Thanks, mom!)  I have had a MacBook Pro since college and now have an iPhone to seamlessly do tasks together. So watching Apple's latest WWDC 2024 was only the next best thing in order to keep track their newest offerings. 

Goal: NLP/Sentiment Analysis on WWDC 2024 - what people liked, what were the most talked about releases, etc.

Performing this analysis will provide insight on how Apple is still in the game since people have been seeing it as behind on the AI front.

In [1]:
!pip install praw duckdb jupysql bertopic groq --quiet

In [2]:
import os
from pprint import pprint

# API's
import googleapiclient.discovery #YouTube
import googleapiclient.errors
import praw # Reddit
from praw.models import MoreComments
from kaggle_secrets import UserSecretsClient

# Data Manipulations
import re
import numpy as np 
import pandas as pd
pd.set_option('display.max_colwidth', None)
import json
from datetime import datetime 

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import duckdb
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

## Generate different embeddings
import tensorflow
import tensorflow_hub as hub

# Topic Modeling
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance

from sklearn.feature_extraction.text import CountVectorizer

# Fast Language Model Inference
from groq import Groq

# Sentiment Analysis
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


2024-06-28 22:53:57.971656: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 22:53:57.971810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 22:53:58.134772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Get the data (YouTube and Reddit)
If there is not enough data, augment data with synonyms

API Links:
* [YouTube](https://console.cloud.google.com/apis/credentials?project=festive-zoo-404500)
* [Reddit](https://ssl.reddit.com/prefs/apps/)

YouTube Videos:
1. https://www.youtube.com/watch?v=qkYoBNdcXBU
1. https://www.youtube.com/watch?v=p2dhZ3AoDDs


Reddit Sub's:
1. https://www.reddit.com/r/apple/comments/1dct23m/wwdc_2024_postevent_megathread/
1. https://www.reddit.com/r/iOSProgramming/comments/1dcmmsm/wwdc_2024_megathread/
1. https://www.reddit.com/r/apple/comments/1de4qkn/what_are_your_biggest_takeaways_from_wwdc_2024_so/

In [3]:
user_secrets = UserSecretsClient()

In [4]:
# YouTube credentials
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = user_secrets.get_secret("youtube_apikey")

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

In [5]:
def getcomments(video):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video,
        maxResults=100
    )

    comments = []

    response = request.execute()

    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([
            comment['authorDisplayName'],
            comment['publishedAt'],
            comment['likeCount'],
            comment['textOriginal'],
            comment['videoId']
        ])

    while (1 == 1):
        try:
            nextPageToken = response['nextPageToken']
        except KeyError:
            break
        nextPageToken = response['nextPageToken']
        # Create a new request object with the next page token.
        nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
        # Execute the next request.
        response = nextRequest.execute()
        # Get the comments from the next response.
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            published_at = comment['publishedAt']
            comments.append([
                comment['authorDisplayName'],
                published_at,
                comment['likeCount'],
                comment['textOriginal'],
                comment['videoId']
            ])

    df0 = pd.DataFrame(comments, columns = ['author', 'published_at', 'like_count', 'text', 'video_id'])
    return df0

In [6]:
# Get all WWDC 2024 comments from YouTube

# vids = [MKBHD, Vox]
vids = ["qkYoBNdcXBU", "p2dhZ3AoDDs"]

df0 = pd.DataFrame()

for v in vids:
    df1 = getcomments(v)
    df0 = pd.concat([df0, df1]) # Combine all df's together into one big dataframe

df0.sample(5)

,author,published_at,like_count,text,video_id
472,@thetpyoutuber7672,2024-06-10T19:25:41Z,10,Are all the freaking features just for iphone 15 pro?,p2dhZ3AoDDs
6012,@justgolfswing6385,2024-06-11T12:17:28Z,0,"@Mkbhbdqwabcdefgh, u think u know more than researchers. \nWhen things are designed better you should respect it. \nThen again, u have to stay in the middle. Subscribers, hey?",qkYoBNdcXBU
1436,@adamcheslyn694,2024-06-12T11:50:40Z,0,"You need to sync up with Elon Musk about Apple and AI. He is freaking out about this on X, and going to ban Apple phones from all of his facilities.",qkYoBNdcXBU
6126,@Shayzare,2024-06-11T12:04:58Z,1,Introducing Apple newest innovation. The Android OS fresh with AI.,qkYoBNdcXBU
1211,@matteouccellini9781,2024-06-12T15:31:34Z,0,"something that was honestly unclear but makes a huge difference: are some of the cool new features that actually use AI in the OS presentations before they talked about actual Apple AI still available on all devices that support the OS, although not of the newest models? For example: iPad calculator with handwritten math, automatic mail sorting, web pages summarisation on Mac ecc?",qkYoBNdcXBU


In [7]:
# Proper date formats
df0['published_at'] = pd.to_datetime(df0['published_at'], format='%Y-%m-%dT%H:%M:%SZ')
df0['published_at_date'] = df0['published_at'].dt.date
df0['published_at_month'] = df0['published_at'].dt.month

In [8]:
# Rename video id's to appropriate video maker
df0['video_id'] = df0['video_id'].map({'qkYoBNdcXBU' : 'MKBHD', 'p2dhZ3AoDDs' : 'The Verge'})
df0.rename(columns = {'video_id':'youtube_channel'}, inplace=True)
df0.sample(5)

,author,published_at,like_count,text,youtube_channel,published_at_date,published_at_month
380,@bloodops3893,2024-06-10 20:24:36,3,"So, google gemini lost the battle about apple 😂",The Verge,2024-06-10,6
9575,@BoxerboyTech,2024-06-11 07:31:47,7,The setup and background is very good 🔥🔥,MKBHD,2024-06-11,6
9637,@warisatilola,2024-06-11 07:29:59,1,I have been waiting for this 🤗🤗😊,MKBHD,2024-06-11,6
3947,@masoncu13,2024-06-11 16:36:49,1,that math thing was dope!!,MKBHD,2024-06-11,6
7526,@pimpel19,2024-06-11 09:30:58,0,"Thanks MKBHD! But one question that bothers me, what about all Apple devices before the A17 Pro Chip? How is Apple considering those regarding the new Apple Intelligence, is Siri gonna stay dumb? Do they adopt some features? I can’t imagine how this works. Me for example with the 14 Pro, having a neural engine on the A16 chip, wouldn’t be able to use those features anyway, because the neural engine is too slow to process on device..? I’d like to know how they plan on doing things in this regard, will they leave all users with older devices behind..",MKBHD,2024-06-11,6


In [9]:
df0['youtube_channel'].value_counts()

youtube_channel
MKBHD        9829
The Verge     483
Name: count, dtype: int64

In [10]:
df0['app'] = 'YouTube'

In [11]:
len(df0)

10312

In [12]:
# Reddit credentials
reddit = praw.Reddit(client_id=user_secrets.get_secret("reddit_client_id"),
                     client_secret=user_secrets.get_secret("reddit_client_secret"),
                     user_agent=user_secrets.get_secret("reddit_user_agent"))

In [13]:
# get all-level Reddit comments for all Reddit posts
def reddit_comments(url):

    all_level_comments = []

    submission = reddit.submission(url=url)

    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        published_at = str(datetime.fromtimestamp(comment.created)) #UTC
        published_at = pd.to_datetime(published_at, format='%Y-%m-%d %H:%M:%S')
        all_level_comments.append({
            "author": comment.author,
            "published_at": published_at, 
            "published_at_date": published_at.date(),
            "published_at_month": published_at.month,
            "like_count": comment.score,
            "text": comment.body
        })

    all_comments_df = pd.DataFrame(all_level_comments)
    return all_comments_df

In [14]:
# Define function to get dataframe for all Reddit comments
def combo_dataframe(links):
    df = pd.DataFrame()
   
    for l in links:
        df2 = reddit_comments(l)
        df = pd.concat([df, df2]) # Combine all df's together into one big dataframe
    return df

In [15]:
# get WWDC 2024 posts' all-level Reddit comments

reddit_links = ['https://www.reddit.com/r/apple/comments/1dct23m/wwdc_2024_postevent_megathread/',
                 'https://www.reddit.com/r/iOSProgramming/comments/1dcmmsm/wwdc_2024_megathread/',
                 'https://www.reddit.com/r/apple/comments/1de4qkn/what_are_your_biggest_takeaways_from_wwdc_2024_so/'] 

reddit_df = combo_dataframe(reddit_links)

In [16]:
reddit_df.head()

author        published_at published_at_date  \
0          BCDragon3000 2024-06-10 18:45:13        2024-06-10   
1            Oulixonder 2024-06-10 18:45:23        2024-06-10   
2       hammerheadtiger 2024-06-10 18:44:49        2024-06-10   
3  ConflictedRedbird186 2024-06-10 18:45:04        2024-06-10   
4       BeefIsForDinner 2024-06-10 18:44:24        2024-06-10   

   published_at_month  like_count  \
0                   6         813   
1                   6         526   
2                   6         876   
3                   6         133   
4                   6          96   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [17]:
len(reddit_df)

1344

In [18]:
reddit_df['app'] = 'Reddit'

In [19]:
# Before concating YouTube and Reddit df's. see if their columns are equal.
list(reddit_df)

['author',
 'published_at',
 'published_at_date',
 'published_at_month',
 'like_count',
 'text',
 'app']

In [20]:
youtube_df = df0.drop(columns=['youtube_channel'], inplace=True)
youtube_df = df0

In [21]:
list(youtube_df)

['author',
 'published_at',
 'like_count',
 'text',
 'published_at_date',
 'published_at_month',
 'app']

In [22]:
# Combine YouTube df and Reddit df together
df = pd.concat([youtube_df, reddit_df])
len(df)

11656

In [23]:
df.head()

,author,published_at,like_count,text,published_at_date,published_at_month,app
0,@rmac02,2024-06-28 21:03:24,0,Bros gobbling on apples knob,2024-06-28,6,YouTube
1,@nignam,2024-06-28 16:20:26,0,Sellout,2024-06-28,6,YouTube
2,@powerplay862,2024-06-28 14:48:30,0,THE MATH NOTE IS PRETTY SICK!\nIT'S ALMOST LIKE CHEATING!!\nGOOD JOB APPLE,2024-06-28,6,YouTube
3,@sufferer21,2024-06-28 14:28:28,0,u still a apple propagandist dude i will never forget this bs from you...,2024-06-28,6,YouTube
4,@jacksmith-mu3ee,2024-06-28 14:20:27,0,Louis rossman destroyed you mate,2024-06-28,6,YouTube


# With only 12,000 samples, augment data using synonyms?

# Perform Bertopic Modeling

In [24]:
# No text pre-processing needed
docs = df['text']

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# All steps together

representation_model = MaximalMarginalRelevance(diversity=0.5)
topic_model = BERTopic(
  embedding_model=embedding_model,    # Step 1 - Extract embeddings
  umap_model=umap_model,              # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words # Diversify topic words
  calculate_probabilities=True,        
  verbose=True,
  representation_model=representation_model
)

topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2024-06-28 22:55:46,403 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/365 [00:00<?, ?it/s]

2024-06-28 22:57:44,980 - BERTopic - Embedding - Completed ✓
2024-06-28 22:57:44,982 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-28 22:58:32,874 - BERTopic - Dimensionality - Completed ✓
2024-06-28 22:58:32,877 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-28 22:58:46,105 - BERTopic - Cluster - Completed ✓
2024-06-28 22:58:46,117 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-28 22:59:08,209 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2944,-1_features_android_ai_ios,"[features, android, ai, ios, google, video, really, screen, icons, phones]","[My biggest Takeaway from all of this is that we’re about to enter a new era of iOS/iPadOS/macOS. We’re finally going to see bumps in ram and processing power and all of these hyper powerful Apple Silicon chips are finally going to be put to actual work. They’ve been impressive so far, But now we’re going to really see what they’re made of and I’m pretty excited for that.\n\nAs a former android diehard user, I’ve come to really appreciate that while Apple “takes a long time“ to implement things that are available on android, all those features are fairly irrelevant in the grand scheme of the stability of the devices.\n\nAnother take away I’m so excited that I ended up getting a 24 GB M3 MacBook Air, instead of the 8 GB model. I had a feeling it would shift this year. \n\nAside from all of that? I’m really excited for RCS messaging. I have a lot of android friends still and I really don’t want to use WhatsApp so we’ve all just been using SMS., I'm really impressed by the continuity features, I'm most of the way into the Apple ecosystem, I use a personal Mac, iPad, Apple TV, and even a work iPhone (plus everyone else in my life has an iPhone so...) \n\nHowever, I've used an Android as my personal phone for nearly 15 years now, and do really enjoy android. And since the majority of the day I'm on my work PC, I like that it has always been a pretty good middle man between the Mac and Windows world. \n\nBut hot damn, just the iPhone Mirroring feature alone makes me think I'm finally going to make the switch to the iPhone side. And there's ton of other really impressive stuff they announced. \nGG Apple., as always, the video i needed for a chill summary of what was actaully important about apple news...100% - looking forward to the iphone 16 launch with ai. it seemed like there was some ai stuff that was useless to most people, and others that were really creative and useful that i would have never thought of. i think wether ai gets to super intelligence or not, it will get better, and at least in the next few years we can expect devices that can connect to other devices and literaly answer emails quickly by voice etc. and make daily life way more efficient by many leaps. the fact apple talked about chat gpt and its connection with them is interesting. but considering, every single major tech company just randomly so happened to release ai software all within the same couple months (more or less) im sure we will be seeing alot more it. the biggest thing i predict is that companies will merge technology and we will live in a city, or state, or possibly world, that all runs off the same network and systems by the same company that uses all the same products etc. so no more apple vs samsung vs whatever. the only devices youll be able to get are the ones that exist which will eventually only be who ever creates the most cost affordable and best product. i understand company diversity creates sustainability i guess but though im sure there will be greed involved in any new societal change as such, i think it would be beneficial. around that time i think a universal basic income is likely also. but such ideas have to be slide in under the rug as not to upset a nation enough to cause issues. anyways im gonna go smoke agian.]"
1,0,996,0_calculator_ipad_handwriting_maths,"[calculator, ipad, handwriting, maths, note, students, high, pencil, homework, lol]","[The calculator 🤣🤣🤣🤣, Math notes🤯, You had me at calculator for iPad😂]"
2,1,536,1_siri_typing_voice_accessibility,"[siri, typing, voice, accessibility, chatgpt, settings, able, ask, new, button]","[AI as in Siri?, What have they done to Siri..., Siri at last.]"
3,2,314,2_ai_apple_tech_marketing,"[ai, apple, tech, marketing, news, naming, rebranding, thing, invented, means]","[AI = Artificial Intelligence ❌❌\nAI = Apple Intel

* The first topic is -1 and contains the most records. This is the outliers topic and should typically be ignored during analysis.

Let's see if different embeddings create more clear topics.


In [25]:
%%time

#load the universal sentence encoder model
use4 = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

#generate embeddings
use4_embeddings = use4(df['text'])
use= np.array(use4_embeddings)

#create list from np arrays to store the embeddings in the dataframe
df['use4'] = use.tolist()

#pass the embeddings into BERTopic
topic_model.fit_transform(docs, use)

#get topic info
topic_model.get_topic_info()

2024-06-28 22:59:41,387 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-28 22:59:55,266 - BERTopic - Dimensionality - Completed ✓
2024-06-28 22:59:55,269 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-28 23:00:05,107 - BERTopic - Cluster - Completed ✓
2024-06-28 23:00:05,119 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-28 23:00:24,093 - BERTopic - Representation - Completed ✓


CPU times: user 2min 5s, sys: 26.8 s, total: 2min 32s
Wall time: 1min 15s


Topic  Count                              Name  \
0       -1   4849         -1_apple_ai_like_features   
1        0    687         0_samsung_s24_ultra_notes   
2        1    569   1_siri_type_accessibility_voice   
3        2    234     2_calculator_app_notes_pencil   
4        3    216        3_naming_ai_chatgpt_insult   
..     ...    ...                               ...   
106    105     17  105_gpt_chat_integration_premium   
107    106     17   106_photos_updates_features_mac   
108    107     17   107_ai_market_buzzword_feedback   
109    108     16                           108____   
110    109     15     109_gai_kitchen_miles_engines   

                                                                               Representation  \
0                           [apple, ai, like, features, android, phone, ios, 15, users, make]   
1                     [samsung, s24, ultra, notes, read, marques, mkbhd, updates, funny, ios]   
2     [siri, type, accessibility, voice, settings, chatgpt, carplay, bixby, button, homepods]   
3             [calculator, app, notes, pencil, students, graphs, feature, ipads, robots, lol]   
4                 [naming, ai, chatgpt, insult, lol, clever, peak, rebranding, appled, brzzy]   
..                                                                                        ...   
106      [gpt, chat, integration, premium, fuckload, device, cancel, generic, free, enhanced]   
107  [photos, updates, features, mac, apps, new, airpods, voice, announcements, productivity]   
108    [ai, market, buzzword, feedback, launched, letters, branding, bs, conflated, pandemic]   
109                                                                      [, , , , , , , , , ]   
110                       [gai, kitchen, miles, engines, theirs, eye, learn, search, hey, 13]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [26]:
topic_model.get_topic(0) # top topic

[('samsung', 0.0913764546566378),
 ('s24', 0.05135511138494995),
 ('ultra', 0.047984932163022534),
 ('notes', 0.03590359332701904),
 ('read', 0.029591736637538092),
 ('marques', 0.029525735677828693),
 ('mkbhd', 0.028744697942352293),
 ('updates', 0.019278260120341353),
 ('funny', 0.014830624944973949),
 ('ios', 0.012662653926299538)]

These topics seem more interpretable due to better embeddings.

#### BERTopic Visualizations 
Reference: https://maartengr.github.io/BERTopic/getting_started/visualization/visualization.html#visualize-probablities-or-distribution

In [27]:
topic_model.visualize_topics()

The biggest topic after the outlier topic (0) was topic 1 which was about Siri. I used the slider to highlight the bubble. When you hover over the bubble, you see the words associated with the topic.

In [28]:
topic_model.visualize_barchart(top_n_topics=8)

Looks like some other huge topics were criticism of Apple being behind in development including the AI frontier. Though a huge win was the showcasing of the interactive/predictive calculator.

In [29]:
topic_model.visualize_heatmap()

* Not sure what the benchmark is for good separation between topics. 
* Considering topics not along the diagonal and not associated with topic 0.  
* Most topics are 0.4 or below in similarity score. This means there are nice distinct topics discovered under better embeddings and BERTopic model.

# Perform Sentiment and Emotion Classification

In [30]:
# using Hugging Face sentiment classifier trained on pre-trained on natural language inference (NLI)
# Info pg: https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student

# Set up the inference pipeline using a model from the 🤗 Hub

sentiment_analysis = pipeline(model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

I need to take care of long comments. Will use an LLM to summarize down to the necessary amount of sentences. Will need to use a Tokenizer to determine max sentences for summary cut-off.

In [31]:
# using uncased (words have no upper or lower case) tokenizer so need to make all words lower case
# normalize text by lowering all letters
df['text'] = df['text'].str.lower()

In [32]:
# Need to make comments text as similar as it was for model training for sentiment inference: remove HTML tags --, expand emojis and slang
# Need to also get rid of weird tags since model was not trained on that for inference
# Remove HTML tags
def cleanhtml(x): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', x) 
    return cleantext.replace('\n\n', ' ').replace('\n', ' ').replace('\t', '')
    
df['processed_text'] = df.apply(lambda row: cleanhtml(row['text']), axis=1)

# # put spaces in between emojis
# def space_emojis(s):
#     return ''.join((' '+c+' ') if c in emoji.UNICODE_EMOJI['en'] else c for c in s)

# df['processed_text'] = df.apply(lambda row: space_emojis(row['processed_text']), axis=1)

# # translating emoji's to English word/slang
# emo = Translator(exact_match_only=False, randomize=True)

# def emoji_trans(x): 
#     return emo.demojify(x)

# df['processed_text'] = df.apply(lambda row: emoji_trans(row['processed_text']), axis=1) # from spaced out emojis into emojis as English words/slang

# Examine sample of dirty comments to see if translation went through
df.head(5)

,author,published_at,like_count,text,published_at_date,published_at_month,app,use4,processed_text
0,@rmac02,2024-06-28 21:03:24,0,bros gobbling on apples knob,2024-06-28,6,YouTube,"[0.05477922037243843, 0.05739085376262665, -0.057579558342695236, 0.03466566652059555, 0.059173405170440674, -0.06090722605586052, 0.013141455128788948, -0.05374596267938614, -0.06175786629319191, 0.04992125928401947, -0.06701109558343887, -0.053923316299915314, -0.05231587216258049, -0.019310804083943367, -0.09462444484233856, -0.05315953120589256, -0.006064518820494413, -0.05956262722611427, 0.03411610424518585, -0.07466941326856613, -0.04750526323914528, 0.06314964592456818, -0.029949961230158806, -0.056469257920980453, -0.06058188155293465, 0.08473767340183258, -0.011972262524068356, 0.050325170159339905, 0.007746669463813305, -0.0002209304948337376, -0.032340142875909805, -0.020155634731054306, -0.04089115560054779, -0.04084071144461632, -0.07144267857074738, 0.053870849311351776, -0.013872222974896431, 0.07515692710876465, -0.048982277512550354, 0.01804482378065586, 0.016927745193243027, -0.03408455103635788, 0.05538281053304672, -0.05294008180499077, -0.0659094974398613, 0.07467073202133179, 0.005771309603005648, -0.04508092626929283, -0.055480584502220154, -0.015562505461275578, 6.442357698688284e-05, -0.038493718951940536, 0.03730695694684982, -0.024685755372047424, -0.06456898152828217, -0.06593665480613708, 0.00540491659194231, 0.02285761572420597, 0.028475819155573845, 0.03391097113490105, 0.05469725653529167, 0.020981773734092712, -0.05814001336693764, 0.05057353526353836, -0.028311334550380707, 0.017940057441592216, 0.03160861134529114, -0.03107207827270031, 0.015672560781240463, -0.05345136299729347, -0.05766218155622482, 0.0014843306271359324, -0.03433616831898689, 0.03097318671643734, 0.03025241196155548, -0.06495372951030731, -0.0031826209742575884, 0.030640998855233192, -0.006702883169054985, -0.009099913761019707, 0.0381338968873024, 0.03157615661621094, 0.03686027601361275, -0.06495168060064316, 0.017992926761507988, 0.02654251642525196, 0.02002965286374092, -0.04300713539123535, 0.06323833018541336, -0.04885079711675644, 0.04157043620944023, -0.04514974728226662, 0.061278991401195526, 0.006096941418945789, 0.004772309213876724, -0.01325955055654049, -0.03772035986185074, 0.028910456225275993, -0.01382940448820591, 0.04684141278266907, ...]",bros gobbling on apples knob
1,@nignam,2024-06-28 16:20:26,0,sellout,2024-06-28,6,YouTube,"[0.04043683782219887, -0.062102310359478, 0.030066149309277534, 0.024012491106987, 0.0419841893017292, 0.032178182154893875, 0.03933561220765114, -0.04045035317540169, -0.000740361399948597, -0.07841067016124725, -0.06352648884057999, 0.07465454936027527, 0.006306588649749756, -0.06371334195137024, -0.02874443680047989, -0.042030975222587585, 0.0387619324028492, -0.03888664022088051, 0.024344077333807945, -0.024612504988908768, -0.05909457057714462, 0.02856578677892685, 0.03158183395862579, -0.030660105869174004, -0.03620532900094986, 0.06799277663230896, 0.02207118272781372, -0.03963794931769371, -0.0028048695530742407, 0.03869693726301193, 0.00813886895775795, -0.02876434102654457, 0.003967220429331064, 0.005340253468602896, 0.026797736063599586, 0.009630238637328148, 0.07118301093578339, -0.0035471287555992603, -0.049167487770318985, 0.0007122739916667342, 0.00839848443865776, 0.013733753934502602, 0.01954125612974167, -0.0750945657491684, 0.004887750837951899, 0.009187932126224041, -0.06003612279891968, 0.08184372633695602, -0.04337496683001518, 0.06912688910961151, 0.08141285181045532, -0.025697143748402596, 0.05083417892456055, 0.016065392643213272, -0.004814372397959232, -0.020158816128969193, -0.003619680181145668, 0.059899184852838516, 0.04908784106373787, 0.03904520347714424, 0.051913417875766754, -0.04419468343257904, -0.022200653329491615, -0.01652652584016323, -0.03392167016863823, -0.017891179770231247, 0.017054114490747452, 0.0021550932433456182, -0.0032344937790185213, 0.033947423100471

In [33]:
# %%time
# Due to transformer error:
# If you are running a transformer model with HuggingFace, there is a chance that one of the input sentences is longer than 512 tokens. 
# Either truncate or split your sentences. I suspect the shorter sentences are padded to 512 tokens.

#1. get token length/total words with max: 500 total words as max total words per comments ince tokenizer can add suffixes as a token
#2. eliminate token lengths greater than max/SUMMARIZE comments using Grok since losing richness of dataset with eliminating comments

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# sentence = "Sentence to check whether it exceeds 512 tokens"

# tokenized_sentence = tokenizer.tokenize(sentence)

# print(len(sentence.split()))  # here is the default length of the sentence
# print(len(tokenized_sentence)) # here is the tokenized length

## Example applied to df

df['num_tokens'] = df.apply(lambda row: len(tokenizer.tokenize(row.processed_text)), axis = 1)
df.head()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (553 > 512). Running this sequence through the model will result in indexing errors


,author,published_at,like_count,text,published_at_date,published_at_month,app,use4,processed_text,num_tokens
0,@rmac02,2024-06-28 21:03:24,0,bros gobbling on apples knob,2024-06-28,6,YouTube,"[0.05477922037243843, 0.05739085376262665, -0.057579558342695236, 0.03466566652059555, 0.059173405170440674, -0.06090722605586052, 0.013141455128788948, -0.05374596267938614, -0.06175786629319191, 0.04992125928401947, -0.06701109558343887, -0.053923316299915314, -0.05231587216258049, -0.019310804083943367, -0.09462444484233856, -0.05315953120589256, -0.006064518820494413, -0.05956262722611427, 0.03411610424518585, -0.07466941326856613, -0.04750526323914528, 0.06314964592456818, -0.029949961230158806, -0.056469257920980453, -0.06058188155293465, 0.08473767340183258, -0.011972262524068356, 0.050325170159339905, 0.007746669463813305, -0.0002209304948337376, -0.032340142875909805, -0.020155634731054306, -0.04089115560054779, -0.04084071144461632, -0.07144267857074738, 0.053870849311351776, -0.013872222974896431, 0.07515692710876465, -0.048982277512550354, 0.01804482378065586, 0.016927745193243027, -0.03408455103635788, 0.05538281053304672, -0.05294008180499077, -0.0659094974398613, 0.07467073202133179, 0.005771309603005648, -0.04508092626929283, -0.055480584502220154, -0.015562505461275578, 6.442357698688284e-05, -0.038493718951940536, 0.03730695694684982, -0.024685755372047424, -0.06456898152828217, -0.06593665480613708, 0.00540491659194231, 0.02285761572420597, 0.028475819155573845, 0.03391097113490105, 0.05469725653529167, 0.020981773734092712, -0.05814001336693764, 0.05057353526353836, -0.028311334550380707, 0.017940057441592216, 0.03160861134529114, -0.03107207827270031, 0.015672560781240463, -0.05345136299729347, -0.05766218155622482, 0.0014843306271359324, -0.03433616831898689, 0.03097318671643734, 0.03025241196155548, -0.06495372951030731, -0.0031826209742575884, 0.030640998855233192, -0.006702883169054985, -0.009099913761019707, 0.0381338968873024, 0.03157615661621094, 0.03686027601361275, -0.06495168060064316, 0.017992926761507988, 0.02654251642525196, 0.02002965286374092, -0.04300713539123535, 0.06323833018541336, -0.04885079711675644, 0.04157043620944023, -0.04514974728226662, 0.061278991401195526, 0.006096941418945789, 0.004772309213876724, -0.01325955055654049, -0.03772035986185074, 0.028910456225275993, -0.01382940448820591, 0.04684141278266907, ...]",bros gobbling on apples knob,6
1,@nignam,2024-06-28 16:20:26,0,sellout,2024-06-28,6,YouTube,"[0.04043683782219887, -0.062102310359478, 0.030066149309277534, 0.024012491106987, 0.0419841893017292, 0.032178182154893875, 0.03933561220765114, -0.04045035317540169, -0.000740361399948597, -0.07841067016124725, -0.06352648884057999, 0.07465454936027527, 0.006306588649749756, -0.06371334195137024, -0.02874443680047989, -0.042030975222587585, 0.0387619324028492, -0.03888664022088051, 0.024344077333807945, -0.024612504988908768, -0.05909457057714462, 0.02856578677892685, 0.03158183395862579, -0.030660105869174004, -0.03620532900094986, 0.06799277663230896, 0.02207118272781372, -0.03963794931769371, -0.0028048695530742407, 0.03869693726301193, 0.00813886895775795, -0.02876434102654457, 0.003967220429331064, 0.005340253468602896, 0.026797736063599586, 0.009630238637328148, 0.07118301093578339, -0.0035471287555992603, -0.049167487770318985, 0.0007122739916667342, 0.00839848443865776, 0.013733753934502602, 0.01954125612974167, -0.0750945657491684, 0.004887750837951899, 0.009187932126224041, -0.06003612279891968, 0.08184372633695602, -0.04337496683001518, 0.06912688910961151, 0.08141285181045532, -0.025697143748402596, 0.05083417892456055, 0.016065392643213272, -0.004814372397959232, -0.020158816128969193, -0.003619680181145668, 0.059899184852838516, 0.04908784106373787, 0.03904520347714424, 0.051913417875766754, -0.04419468343257904, -0.022200653329491615, -0.01652652584016323, -0.03392167016863823, -0.017891179770231247, 0.017054114490747452, 0.0021550932433456182, -0.0032344937790185213, 0.03

In [34]:
# Now let us take a look at comments composition that are around 500 tokens in length

query = """
select author
, num_tokens
, processed_text
from df
order by 2 desc
limit 10
"""

duckdb.query(query).df()

author  num_tokens  \
0    hammerheadtiger        1375   
1      @flowting5163        1246   
2      @mndlessdrwer         763   
3         @ThuanSuBa         707   
4           kasakka1         557   
5       KLM_SpitFire         555   
6   @johnpearson1258         553   
7         @lellmajar         544   
8      @hilal_younus         469   
9  @madboyreadynow28         466   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

469 is the number of tokens closest to 500 and not exceeding it. Let's take a look at the number of sentences that these comments are comprised of.


In [35]:
# Sentence Count
df['sent_count'] = df['processed_text'].str.count('[\w][\.!\?]')

In [36]:
query = """
select author
, num_tokens
, processed_text
, sent_count
from df
order by 2 desc
limit 10
"""

duckdb.query(query).df()

author  num_tokens  \
0    hammerheadtiger        1375   
1      @flowting5163        1246   
2      @mndlessdrwer         763   
3         @ThuanSuBa         707   
4           kasakka1         557   
5       KLM_SpitFire         555   
6   @johnpearson1258         553   
7         @lellmajar         544   
8      @hilal_younus         469   
9  @madboyreadynow28         466   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

## Try to use open source LLM/Fast Language Model (groqcloud) to summarize Pandas text column
Reference: https://console.groq.com/docs/quickstart

In [37]:
client = Groq(
    #api_key=os.environ.get(user_secrets.get_secret("groq_key")),
    api_key=user_secrets.get_secret("groq_key"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": '''
            
            Please summarize the following comment in 15 sentences or less as if you are the author trying to write a condensed version of your original comment.
            Here is the original comment:
            i want to thank you for always giving us a well thought videos. others spend time babbling and trying to be funny and you sound like a tech guy giving us tech news. i have a suggestion to take you to a new level. i have been writing apple for years. i honestly believe they have used some of my ideas. i’ll share them in a moment. by the way i am a software engineer. i am not one who thinks i am so d smart or the smarted person in the room. i love to always learn and ai find i am always learning new things from you. so my idea for you is a real tech news channel. this will make you something like the wal mart of tech news. this idea could put some tech influencers out of business with this idea. i tried to float this to apple but they don’t seem to like the idea. we have cnn, nbc news, abc news, cbs news and from time to time we get tech news but mostly when something new like a new iphone is released. tech news is happening every day all day. i say join forces with some of the best like yourself, brian tong, possibly some from engadget and create a real tech news channel. it’s not based around politics although t today politics and tech seem to collide. evs and republicans wanting to k evs, tictok, twitter. with social media companies being as influential as they have become. they’re not on the hot seat enough with cameras in their face or reporters running up on them to ask difficult questions. a real tech news channel will do this. what you and others do is journalism. i want the news to still be fun but i want it just like real news every day 24/7 just like a cnn. you will hire anchors do some in-depth interviews like we see you do often with tech ceo’s. it’s a militia billion. dollar idea i am giving to you. this is something new of its kind and needed. tech is no longer just mobile. we have stepped into the world of ai and humans need to understand how these new pieces of tech will i pack them. i would love to work with you as a producer or something. i have a huge idea and it will work.	31
            
            ''',
        }
    ],
    model="llama3-8b-8192",
)

chat_completion.choices[0].message.content

"Here is a condensed version of the original comment:\n\nI want to thank you for creating well-researched and informative videos. As a software engineer, I appreciate your tech-heavy content and find it both enjoyable and educational. I have a suggestion to take your content to the next level: creating a 24/7 real tech news channel. This would differentiate you from other tech influencers and put some out of business. I've been writing to Apple for years and believe they've used some of my ideas. I'd love to collaborate with you and others like Brian Tong and Engadget to make this channel a reality. The idea would focus on tech news, not politics, and would be a departure from the typical tech news we see today. We could hire anchors and conduct in-depth interviews with tech CEOs. I believe this has the potential to be a billion-dollar idea and would love to work with you as a producer."

In [38]:
# Exclude chat bot intro portion as whole inference
spl_word = '\n\n'
res = chat_completion.choices[0].message.content.split(spl_word, 1)
res[1]

"I want to thank you for creating well-researched and informative videos. As a software engineer, I appreciate your tech-heavy content and find it both enjoyable and educational. I have a suggestion to take your content to the next level: creating a 24/7 real tech news channel. This would differentiate you from other tech influencers and put some out of business. I've been writing to Apple for years and believe they've used some of my ideas. I'd love to collaborate with you and others like Brian Tong and Engadget to make this channel a reality. The idea would focus on tech news, not politics, and would be a departure from the typical tech news we see today. We could hire anchors and conduct in-depth interviews with tech CEOs. I believe this has the potential to be a billion-dollar idea and would love to work with you as a producer."

In [39]:
#Let's check if this condensed version from Groq is <512/500 tokens in length
len(tokenizer.tokenize(res[1]))

182

In [40]:
# Now write up a function in order to loop through Pandas text column that has >500 tokens 
# Put responses in a list
# Add summaries to df 
# Now have run tokenizer
# Check to make sure that each row now has 500 or less tokens
# After verification, perform sentiment analysis
# Possibly perform sentiment analysis using HF and Groq

In [41]:
# %%time

# df = (
# df.assign(sentiment = lambda x: x['text'].apply(lambda s: sentiment_analysis(s)))
#     .assign(
#          label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
#          score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
#     )
# )

# df.head()

# Aspect Based Sentiment Analysis (ASBA) Using PyABSA

# Named Entity Recognition in order to isolate products?

# Radar Graph between 2 similar, close in proximity restaurants 